In [1]:
# https://hoit1302.tistory.com/159

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [3]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
import os
import re
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import transformers
from transformers import AdamW
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import Trainer
from transformers import TrainingArguments

import datasets
from datasets import load_dataset, load_metric, ClassLabel

import huggingface_hub
import pyarrow

import torch
torch.backends.cuda.matmul.allow_tf32 = True

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.schedulers.pb2 import PB2
from ray.tune.examples.pbt_transformers.utils import (
    download_data,
    build_compute_metrics_fn,
)

from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict
from typing import TypeVar, Type
from numba import cuda
import GPy

In [6]:
import transformers
import datasets
import huggingface_hub
import pyarrow
import torch
print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(pyarrow.__version__)
print(torch.__version__)

4.8.1
1.11.0
0.0.12
9.0.0
1.9.0+cu111


In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    device_count = torch.cuda.device_count()
    print("device_count: {}".format(device_count))
    for device_num in range(device_count):
        print("device {} capability {}".format(
            device_num,
            torch.cuda.get_device_capability(device_num)))
        print("device {} name {}".format(
            device_num, 
            torch.cuda.get_device_name(device_num)))
else:
    device = torch.device("cpu")
    print("no cuda device")

device_count: 1
device 0 capability (8, 6)
device 0 name NVIDIA GeForce RTX 3080


In [8]:
#### The number of CPU cores
!grep -c processor /proc/cpuinfo

20


In [9]:
# import bert_functions
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [10]:
def preprocessing(dataset: DatasetDict,
                  text_column: str,
                  label_column: str,
                  id_column: str,
                  model_name: str,
                  train_proportion: float,
                  seed: int,
                  custom_tokenizer_dir: str = "my_result"
                  ) -> tuple:
    if not isinstance(dataset, DatasetDict):
        raise TypeError(f"Values in `dataset` should be of type `DatasetDict` but got type '{type(dataset)}'")
    
    # Select columns to use
    print("Removing rows with missing value...")
    cols_to_remove = list(dataset['train'].features.keys())
    cols_to_remove.remove(id_column)
    cols_to_remove.remove(text_column)
    cols_to_remove.remove(label_column)
    dataset = dataset.remove_columns(cols_to_remove)
    if 'text' not in dataset['train'].features.keys():
        dataset = dataset.rename_column(text_column, "text")
    if label_column not in dataset['train'].features.keys():
        dataset = dataset.rename_column(label_column, "label")
    if id_column not in dataset['train'].features.keys():
        dataset = dataset.rename_column(id_column, "id")
     
    # Remove NA rows
    dataset = dataset.filter(lambda row: pd.notnull(row["text"]))
    print("Done. (1/4)")
    
    # Remove specal characters
    print("Removing special characters...")
    def remove_sp_fn(dataset):
        dataset["text"]=re.sub(r'[^a-z|A-Z|0-9|ㄱ-ㅎ|ㅏ-ㅣ|가-힣| ]+', '', str(dataset["text"]))
        return dataset
    
    dataset = dataset.map(remove_sp_fn)
    print("Done. (2/4)")
    
    # Tokenize
    print("Tokenining the text column...")
    tokenizer = KoBERTTokenizer.from_pretrained(model_name, truncation_side = 'right')
    def tokenize_fn(dataset):
        tokenized_batch = tokenizer(dataset["text"], truncation=True, padding="max_length", max_length=256) # max_length=24??
        return tokenized_batch
    
    dataset = dataset.map(tokenize_fn, batched=True)
    tokenizer.save_pretrained(custom_tokenizer_dir)
    print("Done. (3/4)")
    
    # train-evaluation-test split
    print("Spliting train-evaluation-test set...")
    train_dataset = dataset["train"].shuffle(seed=seed).select(range(0,math.floor(len(dataset["train"])*train_proportion)))
    eval_dataset = dataset["train"].shuffle(seed=seed).select(range(math.floor(len(dataset["train"])*train_proportion), len(dataset["train"])))
    test_dataset = dataset["test"]
    print("Done. (4/4)")
    
    return train_dataset, eval_dataset, test_dataset

In [11]:
def modeling(train_dataset: Dataset,
             eval_dataset: Dataset,
             num_labels: int, 
             model_name: str,
             num_gpus: int,
             num_cpus: int,
             seed: int,
             output_dir: str = './output',
             logging_dir: str = "./logs",
             do_hpo: bool = False,
             std: float = 0.1,
             n_trials: int = 5,
             patience: int = 3,
             hpo_result_dir: str = "./hpo-results",
             hpo_result_dir_subfolder_name: str = 'tune_transformer_pbt',
             custom_model_dir: str = "my_result"
             ) -> Type[Trainer]:
    if not isinstance(train_dataset, Dataset):
        raise TypeError(f"Values in `train_dataset` should be of type `Dataset` but got type '{type(train_dataset)}'")
    elif not isinstance(eval_dataset, Dataset):
        raise TypeError(f"Values in `eval_dataset` should be of type `Dataset` but got type '{type(eval_dataset)}'")
        
    train_dataset = train_dataset.remove_columns('id')
    eval_dataset = eval_dataset.remove_columns('id')
        
    # Load the model 
    def _model_init():
        return AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels = 7,
            output_attentions = False,
            output_hidden_states = False
            )

    # Define metrics to use for evaluation
    def _compute_metrics(eval_pred):
#         metric1 = load_metric("accuracy")
#         metric2 = load_metric("f1")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
#         accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
#         f1 = metric2.compute(predictions=predictions, references=labels)["f1"]
        f1_weighted = f1_score(labels, predictions, average = 'weighted')
        acc_weighted = accuracy_score(labels, predictions)
        return {"acc_weighted": acc_weighted, "f1_weighted": f1_weighted, "objective": acc_weighted + f1_weighted}

    # Default: batch size = 32, evaluate every 50 steps
    training_args = TrainingArguments(
        output_dir=output_dir,
        do_train=True,
        do_eval=True,
        evaluation_strategy="steps",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=4,
        learning_rate=2e-5, # config
        weight_decay=0.1, # config
        adam_beta1=0.9, # config
        adam_beta2=0.9, # config
        adam_epsilon=1.5e-06, # config
        num_train_epochs=20, # config
        max_steps=-1,
        lr_scheduler_type="linear",
        warmup_ratio=0.1,  # config
        warmup_steps=0,
        logging_dir=logging_dir,
        save_strategy="steps",
        no_cuda=num_gpus <= 0, 
        seed=seed,  # config
#         bf16=False, # Need torch>=1.10, Ampere GPU with cuda>=11.0
        fp16=True,
#         tf32=True, 
        eval_steps = 50,
        load_best_model_at_end=True,
        greater_is_better=True,
        metric_for_best_model="objective", # f1 + acc
        report_to="none",
        skip_memory_metrics=True,
#         gradient_checkpointing=True
        )
    
    # Calculate class weights
    train_labels = np.array(train_dataset["label"])
    class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)
    weights = torch.tensor(class_weights, dtype = torch.float)
    
    # Define device
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
    else: 
        device = torch.device("cpu")

    # Customize trainer class to apply class weights
    class CustomTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            labels = inputs.get("labels")
            # forward pass
            outputs = model(**inputs)
            logits = outputs.get("logits")
            # compute custom loss
            weight = weights.to(device)
#             loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
            loss_fct = torch.nn.MultiMarginLoss(weight=weight)
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            return (loss, outputs) if return_outputs else loss

    trainer = CustomTrainer(
        model_init=_model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=_compute_metrics,
        )
    
    if do_hpo == True:
    
        # Initialize Ray
        ray.shutdown()
        ray.init(log_to_driver=False, ignore_reinit_error=True, num_cpus=num_cpus, num_gpus=num_gpus, include_dashboard=False)

        # Fix batch_size in each trial
        def _hp_space(trial):
            return {
                "per_device_train_batch_size": 16,
                "per_device_eval_batch_size": 32,
                "max_steps": -1,
                "weight_decay": tune.uniform(0.0, 0.1),
                "learning_rate": tune.uniform(1e-6, 1e-4),
                "warmup_ratio": tune.uniform(0.0, 0.1),
                "adam_beta1": tune.loguniform(0.97, 0.99),
                "adam_beta2": tune.loguniform(0.97, 0.99),
                "adam_epsilon": tune.loguniform(1e-9, 1e-7),
                "max_grad_norm": tune.uniform(0.0, 5.0),
            }

#         def explore(config):
#             # Define your hyperparameters' ranges.
#             bounds = {
#                 "weight_decay": (0.0, 0.1),
#                 "learning_rate": (1e-6, 1e-4),
#                 "warmup_ratio": (0.0, 0.1),
#                 "adam_beta1": (0.97, 0.99),
#                 "adam_beta2": (0.97, 0.99),
#                 "adam_epsilon": (1e-9, 1e-7),
#                 "max_grad_norm": (0.0, 5.0),
#             }

#             factor = 0.2  # The perturbation factor.

#             for hyperparam, (lower, upper) in bounds.items():
#                 value = config[hyperparam]

#                 if np.random.uniform() > 0.5:  # Apply perturbation.
#                     scale_factor = 1 + factor if np.random.uniform() > 0.5 else 1 - factor
#                     value *= scale_factor
#                 else:  # Resample.
#                     value = np.random.uniform(lower, upper)

#                 # Clip to ensure it's within bounds.
#                 config[hyperparam] = np.clip(value, lower, upper)
            
#             return config

#         # PBT schduler
#         scheduler = PopulationBasedTraining(
#             time_attr="training_iteration",
#             metric="objective",
#             mode="max",
#             perturbation_interval=1,
#             custom_explore_fn=explore
#         )

        # PB2 schduler
        scheduler = PB2(
            time_attr="training_iteration",
            metric="objective",
            mode="max",
            perturbation_interval=60, # default = 60
            hyperparam_bounds={
                "weight_decay": [0.0, 0.1],
                "learning_rate": [1e-6, 1e-4],
                "warmup_ratio": [0.0, 0.1],
                "adam_beta1": [0.97, 0.99],
                "adam_beta2": [0.97, 0.99],
                "adam_epsilon": [1e-9, 1e-7],
                "max_grad_norm": [0.0, 5.0],
            }
        )

        # Define columns to report
        reporter = CLIReporter(
            parameter_columns={
                "learning_rate": "lr",
                "warmup_ratio" : "warmup_ratio",
                "max_grad_norm" : "max_grad_norm"
            },
            metric_columns=["eval_acc_weighted", "eval_f1_weighted", "eval_objective", "eval_loss", "epoch", "training_iteration"]
        )

        # Early stopping
        stopper = tune.stopper.ExperimentPlateauStopper(metric="objective", 
                                                        std=std,
                                                        top=n_trials,
                                                        mode="max",
                                                        patience=patience
                                                        )

        # HPO
        hpo_result = trainer.hyperparameter_search(
            hp_space = _hp_space,
            direction = "maximize",
            backend="ray",
            reuse_actors = True,
            n_trials=n_trials,
            resources_per_trial={"cpu": num_cpus, "gpu": num_gpus},
            scheduler=scheduler,
            keep_checkpoints_num=1,
            checkpoint_score_attr="training_iteration",
            stop=stopper,
            progress_reporter=reporter,
            local_dir=hpo_result_dir,
            name=hpo_result_dir_subfolder_name,
            log_to_file=True,
        )
        for n, v in hpo_result.hyperparameters.items():
            setattr(trainer.args, n, v)
    else:
        pass
    
    return trainer

In [12]:
def evaluation(trainer: Trainer, 
               eval_dataset: Dataset,
               text_column_name: str
               ) -> pd.DataFrame:
    if not isinstance(eval_dataset, Dataset):
        raise TypeError(f"Values in `eval_dataset` should be of type `Dataset` but got type '{type(eval_dataset)}'")
        
    # id column
    eval_dataset_id = eval_dataset
    eval_dataset_id = eval_dataset_id.remove_columns(['text', 'label', 'input_ids', 'attention_mask', 'token_type_ids'])
    eval_dataset_id = eval_dataset_id.to_pandas()
    
    # Add ID to the result after performing prediction with eval data
    eval_dataset = eval_dataset.remove_columns('id')
    eval_pred_result = trainer.predict(test_dataset=eval_dataset)
    
    # prediction result
    pred_df = pd.DataFrame(eval_pred_result.predictions)
    pred_df.columns = [f'{text_column_name}_pred_0', f'{text_column_name}_pred_1', f'{text_column_name}_pred_2', f'{text_column_name}_pred_3', f'{text_column_name}_pred_4', f'{text_column_name}_pred_5', f'{text_column_name}_pred_6']
    
    # classification result
    cls_label = list(eval_pred_result.label_ids)
    cls_pred = list(map(lambda x: x.index(max(x)), eval_pred_result.predictions.tolist()))
    
    eval_result_df = pd.concat([pd.DataFrame(eval_dataset_id),
                                pd.DataFrame(eval_dataset['text']), 
                                pd.DataFrame(pred_df), 
                                pd.DataFrame(cls_label),
                                pd.DataFrame(cls_pred)],
                               axis=1)
                               
    eval_result_df.columns = ['id', 'text', f'{text_column_name}_pred_0', f'{text_column_name}_pred_1', f'{text_column_name}_pred_2', f'{text_column_name}_pred_3', f'{text_column_name}_pred_4', f'{text_column_name}_pred_5', f'{text_column_name}_pred_6', 'label', 'pred']
    
    return eval_result_df

In [13]:
# hardware
num_cpus = 16
num_gpus = 1

# model
model_name = "skt/kobert-base-v1"

# data
text_column = 'text' 
label_column = 'label'
id_column = 'id'
custom_dir = "SA_model"

# process
seed = 9572
train_proportion = 0.8
do_hpo = True
# If you want to search best hyperparameters using ray tune(do_hpo = True), parameters below should be set for ealry sttoping
n_trials = 5 # 새로운 experiment 수행 시마다 n_trials개의 최고 성능을 가진 experiment을 유지
std = 0.01 # metric의 표준 편차. metric의 표준편차가 std보다 작아지면, n_trials 개의 최고 성능을 가진 결과를 유지하면서 patience번째 epoch에서 실험을 중지
patience = 5 # metric이 std보다 작은 상태로 patience epoch 동안 개선되지 않으면 Stopper 활성화

In [14]:
data_name = "sent_merge" 

data_dir = f'./{data_name}.csv' # your local raw data dir
data = pd.read_csv(data_dir)
data.rename(columns = {'Unnamed: 0':'id'}, inplace=True)
data

,id,label,text
0,1,기쁨,ㅅ 두둠칫 두둠칫 | | | | | | L
1,2,우울,부모님께짐만되는거 같아요
2,3,분노,그 옛날 액티브 엑스 찬양하던 새끼들 어디갔어
3,4,두려움,밖에나가는게 좀 무섭다라고할까요
4,5,분노,욕 밖에 안나온다
...,...,...,...
54812,54876,분노,힘있고 돈많으면 굽실굽실 힘없고 가난하면 으쓱으쓱
54813,54877,분노,힘좀세졋다고 천조국한테까지 깝치지만 현실은 힘만센 속좁은 찌질이
54814,54878,두려움,힘주면 옆에 뽝 서는게 뭐죠
54815,54879,우울,힝 불타는 금요일인데 맥주한잔을 못하다니ㅠㅠㅜ 감기 주거버렷ㅠㅜ


In [15]:
data['label'] = data['label'].replace(['기쁨', '우울', '분노', '두려움', '사랑', '놀람', '중립'], [0,1,2,3,4,5,6])

In [16]:
# y = data.label
# X = data.drop('label', axis=1)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=seed) # The test set is 20% of the total as default.

# train = pd.concat([X_train, y_train], axis=1)
# test = pd.concat([X_test, y_test], axis=1)

# train.to_csv(f'../data_split/{data_name}_train.csv', index=False) # your local splited data dir
# test.to_csv(f'../data_split/{data_name}_test.csv', index=False) # your local splited data dir

In [17]:
dataset = load_dataset('csv', data_files={'train': f'../data_split/{data_name}_train.csv',
                                          'test': f'../data_split/{data_name}_test.csv'})
dataset

Using custom data configuration default-8e80763ffbac1bee
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-8e80763ffbac1bee/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 43853
    })
    test: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 10964
    })
})

In [18]:
train_dataset, eval_dataset, test_dataset = preprocessing(dataset = dataset,
                                                          text_column = text_column, 
                                                          label_column = label_column,
                                                          id_column = id_column,
                                                          model_name = model_name,
                                                          train_proportion = train_proportion,
                                                          seed = seed,
                                                          custom_tokenizer_dir = custom_dir)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8e80763ffbac1bee/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-b91a2ee2b4f71682.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8e80763ffbac1bee/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-04f2abd210e34614.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8e80763ffbac1bee/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-12a6d097444e3a61.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-8e80763ffbac1bee/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-6983a70bf50cc5be.arrow


Removing rows with missing value...
Done. (1/4)
Removing special characters...
Done. (2/4)
Tokenining the text column...


  0%|          | 0/44 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-8e80763ffbac1bee/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-c6b36cc81a7eeb62.arrow


Done. (3/4)
Spliting train-evaluation-test set...
Done. (4/4)


In [19]:
trainer = modeling(train_dataset=train_dataset,
                   eval_dataset=eval_dataset,
                   num_labels = 7,
                   model_name=model_name,
                   num_gpus=num_gpus,
                   num_cpus=num_cpus,
                   seed=seed,
                   output_dir='./output',
                   logging_dir="./logs",
                   do_hpo=do_hpo,
                   std = std,
                   n_trials = n_trials,
                   patience = patience,
                   hpo_result_dir = "./hpo-results",
                   hpo_result_dir_subfolder_name = 'tune_transformer_pbt')

loading configuration file https://huggingface.co/skt/kobert-base-v1/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/b8898dabd49ed32401ee6a6bc5eb011f12728750b44d08b151acf270bf1732ca.1007ab583c49854e3c65c61288a980ae4d25a4bbfa51b51915ec1772f02f992d
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "kobert_version": 1.0,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mode

== Status ==
Current time: 2023-06-01 06:23:12 (running for 00:00:00.10)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/playdoh_SA/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-------------+----------------+-----------------+
| Trial name             | status   | loc                |          lr |   warmup_ratio |   max_grad_norm |
|------------------------+----------+--------------------+-------------+----------------+-----------------|
| _objective_c90f4_00000 | RUNNING  | 172.17.0.3:1626981 | 3.59352e-05 |      0.0796373 |         1.55334 |
| _objective_c90f4_00001 | PENDING  |                    | 7.73268e-05 |      0.0476358 |         1.25659 |
| _objective_c90f4_00002 | PENDING  |                    | 7.95906e-05 |      0.0994581 |         2.10915 |
| _objective_c90f4_00003 | PEND

== Status ==
Current time: 2023-06-01 06:23:48 (running for 00:00:36.26)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/playdoh_SA/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-------------+----------------+-----------------+
| Trial name             | status   | loc                |          lr |   warmup_ratio |   max_grad_norm |
|------------------------+----------+--------------------+-------------+----------------+-----------------|
| _objective_c90f4_00000 | RUNNING  | 172.17.0.3:1626981 | 3.59352e-05 |      0.0796373 |         1.55334 |
| _objective_c90f4_00001 | PENDING  |                    | 7.73268e-05 |      0.0476358 |         1.25659 |
| _objective_c90f4_00002 | PENDING  |                    | 7.95906e-05 |      0.0994581 |         2.10915 |
| _objective_c90f4_00003 | PEND

Trial name,date,done,eval_acc_weighted,eval_f1_weighted,eval_loss,eval_objective,hostname,iterations_since_restore,node_ip,objective,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
_objective_c90f4_00000,2023-06-01_06-51-49,True,0.639722,0.648873,0.644102,1.2886,ff57561168b8,38,172.17.0.3,2.57719,1626981,True,1715.97,45.0017,1715.97,1685602309,38,c90f4_00000


== Status ==
Current time: 2023-06-01 06:24:08 (running for 00:00:55.80)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/playdoh_SA/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------+
| Trial name             | status   | loc                |          lr |   warmup_ratio |   max_grad_norm |   eval_acc_weighted |   eval_f1_weighted |   eval_objective |   eval_loss |   training_iteration |
|------------------------+----------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------|
| _objective_c90f4_00000 | RUNNING  | 172.17.0.3:1626981 |

...

== Status ==
Current time: 2023-06-01 06:51:04 (running for 00:27:52.30)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/playdoh_SA/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------+
| Trial name             | status   | loc                |          lr |   warmup_ratio |   max_grad_norm |   eval_acc_weighted |   eval_f1_weighted |   eval_objective |   eval_loss |   training_iteration |
|------------------------+----------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------|
| _objective_c90f4_00000 | RUNNING  | 172.17.0.3:1626981 |

== Status ==
Current time: 2023-06-01 06:51:24 (running for 00:28:12.31)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/playdoh_SA/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------+
| Trial name             | status   | loc                |          lr |   warmup_ratio |   max_grad_norm |   eval_acc_weighted |   eval_f1_weighted |   eval_objective |   eval_loss |   training_iteration |
|------------------------+----------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------|
| _objective_c90f4_00000 | RUNNING  | 172.17.0.3:1626981 |

== Status ==
Current time: 2023-06-01 06:51:44 (running for 00:28:32.35)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 16.0/16 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/playdoh_SA/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------+
| Trial name             | status   | loc                |          lr |   warmup_ratio |   max_grad_norm |   eval_acc_weighted |   eval_f1_weighted |   eval_objective |   eval_loss |   training_iteration |
|------------------------+----------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------|
| _objective_c90f4_00000 | RUNNING  | 172.17.0.3:1626981 |

2023-06-01 06:51:49,865	INFO tune.py:945 -- Total run time: 1717.36 seconds (1717.23 seconds for the tuning loop).


== Status ==
Current time: 2023-06-01 06:51:49 (running for 00:28:37.23)
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Logical resource usage: 0/16 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/playdoh_SA/hpo-results/tune_transformer_pbt
Number of trials: 5/5 (5 TERMINATED)
+------------------------+------------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------+
| Trial name             | status     | loc                |          lr |   warmup_ratio |   max_grad_norm |   eval_acc_weighted |   eval_f1_weighted |   eval_objective |   eval_loss |   training_iteration |
|------------------------+------------+--------------------+-------------+----------------+-----------------+---------------------+--------------------+------------------+-------------+----------------------|
| _objective_c90f4_00000 | TERMINATED | 172.17.0.3:1626981 | 3.59

In [ ]:
trainer.train()

loading configuration file https://huggingface.co/skt/kobert-base-v1/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/b8898dabd49ed32401ee6a6bc5eb011f12728750b44d08b151acf270bf1732ca.1007ab583c49854e3c65c61288a980ae4d25a4bbfa51b51915ec1772f02f992d
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "kobert_version": 1.0,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "mode

Step,Training Loss,Validation Loss,Acc Weighted,F1 Weighted,Objective
50,No log,0.850977,0.262228,0.185422,0.447650
100,No log,0.831618,0.310683,0.236585,0.547268
150,No log,0.858357,0.331091,0.205266,0.536357
200,No log,0.871822,0.365181,0.298699,0.663880
250,No log,0.806623,0.394710,0.308978,0.703688
300,No log,0.744923,0.424695,0.369367,0.794062
350,No log,0.666028,0.516361,0.511211,1.027571
400,No log,0.651724,0.538821,0.522631,1.061452
450,No log,0.580038,0.549652,0.543652,1.093304
500,0.772600,0.652914,0.529700,0.540412,1.070113


 /opt/conda/lib/python3.8/site-packages/transformers/trainer.py:1297: FutureWarning:Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-50
Configuration saved in ./output/checkpoint-50/config.json
Model weights saved in ./output/checkpoint-50/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Sa

Model weights saved in ./output/checkpoint-900/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-950
Configuration saved in ./output/checkpoint-950/config.json
Model weights saved in ./output/checkpoint-950/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-1000
Configuration saved in ./output/checkpoint-1000/config.json
Model weights saved in ./output/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have b

***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-1950
Configuration saved in ./output/checkpoint-1950/config.json
Model weights saved in ./output/checkpoint-1950/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-2000
Configuration saved in ./output/checkpoint-2000/config.json
Model weights saved in ./output/checkpoint-2000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-2050
Configuration saved in ./output/checkpoint-2050/config.json
Model weights

Model weights saved in ./output/checkpoint-2950/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-3000
Configuration saved in ./output/checkpoint-3000/config.json
Model weights saved in ./output/checkpoint-3000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-3050
Configuration saved in ./output/checkpoint-3050/config.json
Model weights saved in ./output/checkpoint-3050/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and ha

***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-3900
Configuration saved in ./output/checkpoint-3900/config.json
Model weights saved in ./output/checkpoint-3900/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-3950
Configuration saved in ./output/checkpoint-3950/config.json
Model weights saved in ./output/checkpoint-3950/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-4000
Configuration saved in ./output/checkpoint-4000/config.json
Model weights

Model weights saved in ./output/checkpoint-4900/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-4950
Configuration saved in ./output/checkpoint-4950/config.json
Model weights saved in ./output/checkpoint-4950/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8771
  Batch size = 32
Saving model checkpoint to ./output/checkpoint-5000
Configuration saved in ./output/checkpoint-5000/config.json
Model weights saved in ./output/checkpoint-5000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and ha

In [ ]:
# save the pretrained model
trainer.model.save_pretrained(custom_dir)

In [ ]:
# load model
ft_model_path = './SA_model'
loaded_model = AutoModelForSequenceClassification.from_pretrained(ft_model_path)
trainer = Trainer(model=loaded_model)

In [ ]:
df = evaluation(trainer = trainer,
                eval_dataset = eval_dataset,
                text_column_name = text_column)

In [ ]:
result_df = df.copy()
result_df.columns = ["id", "text", "기쁨", "우울", '분노', '두려움', '사랑', '놀람', '중립', 'label', 'pred']

def softmax(row):
    e = np.exp(row) 
    return e / e.sum()

cols = ["기쁨", "우울", '분노', '두려움', '사랑', '놀람', '중립']
result_df[cols] = result_df[cols].apply(softmax, axis=1)

result_df['label'] = result_df['label'].replace([0,1,2,3,4,5,6], ['기쁨', '우울', '분노', '두려움', '사랑', '놀람', '중립'])
result_df['pred'] = result_df['pred'].replace([0,1,2,3,4,5,6], ['기쁨', '우울', '분노', '두려움', '사랑', '놀람', '중립'])

In [ ]:
result_df.sample(10)

In [ ]:
# # 나눔글꼴 설치
# !apt-get install -y fonts-nanum
# # matplotlib에 남아있는 폰트 캐시 삭제
# !rm -rf ~/.cache/matplotlib/*
# !fc-cache -fv

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False

# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'NanumGothic'

In [ ]:
import matplotlib.pyplot as plt

text_index = 192

# Select the row (for example, the first row)
row = result_df.loc[text_index, cols]

# Create a horizontal bar plot
plt.figure(figsize=(10, 5))  # Set the figure size
plt.barh(cols, row, color='skyblue')  # Create a horizontal bar plot
plt.xlabel('Percentage')  # Set the x-label
plt.title('Prediction Percentages for Text 0')  # Set the title
plt.show()  # Display the plot

print(result_df['text'][text_index])

In [ ]:
# Check the classification result of each XLM-RoBERTa Model 
from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)

print("Confusion Matrix")
print(confusion_matrix(df.label, df.pred))
print("-------------------------")
accuracy = accuracy_score(df.label, df.pred)
f1 = f1_score(df.label, df.pred, average='weighted')
recall = recall_score(df.label, df.pred, average='weighted')
precision = precision_score(df.label, df.pred, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 score: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")

In [ ]:
def evaluation_text(text: str,
                    model_dir: str,
                    truncation_side = 'right'):
    
    labels = {0: '기쁨', 1: '우울', 2: '분노', 3: '두려움', 4: '사랑', 5: '놀람', 6: '중립'}
    
    model= AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 7)
    tokenizer = KoBERTTokenizer.from_pretrained(model_name, truncation_side = truncation_side)
    
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    output = model(**inputs)
    logits = output[0]
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    probs = probabilities.detach().cpu().numpy()

    # Convert the probabilities to a DataFrame
    df = pd.DataFrame(probs, columns=labels.values())

    return df

text = "어제 남자친구를 사귀었다 그럼에도 불구하고 우울은 끝없이 증가하며 나는 그 사이에 있다 잘 한걸까잘못한걸까"
print(evaluation_text(text, model_dir = custom_dir))